In [1]:
#----------------------------------------------------------------------
# Purpose:  Condition an Airline dataset by filtering out NAs where the
#           departure delay in the input dataset is unknown.
#
#           Then treat anything longer than minutesOfDelayWeTolerate
#           as delayed.
#----------------------------------------------------------------------

In [2]:
import sys
sys.path.insert(1, '/Users/rpeck/Source/h2o-3/h2o-py')
import h2o

In [4]:
h2o.init(ip="127.0.0.1", port=54321, strict_version_check=False)

H2O cluster uptime:,48 minutes 55 seconds 95 milliseconds
H2O cluster version:,3.7.0.99999
H2O cluster name:,wendycwong
H2O cluster total nodes:,1
H2O cluster total memory:,3.56 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [5]:
air = h2o.import_frame(h2o.locate("/Users/wendycwong/h2o-3/smalldata/airlines/allyears2k_headers.zip"))


Parse Progress: [##################################################] 100%


In [6]:
numRows, numCols = air.dim
print "Original dataset rows: {0}, columns: {1}".format(numRows, numCols)

x_cols = ["Month", "DayofMonth", "DayOfWeek", "CRSDepTime", "CRSArrTime", "UniqueCarrier", "CRSElapsedTime", "Origin", "Dest", "Distance"]
y_col = "SynthDepDelayed"

noDepDelayedNAs = air[air["DepDelay"].isna() == 0]
rows, cols = noDepDelayedNAs.dim
print "New dataset rows: {0}, columns: {1}".format(rows, cols)

NameError: name 'air' is not defined

In [7]:
minutesOfDelayWeTolerate = 15
noDepDelayedNAs = noDepDelayedNAs.cbind(noDepDelayedNAs["DepDelay"] > minutesOfDelayWeTolerate)
noDepDelayedNAs[numCols] = noDepDelayedNAs[numCols-1].asfactor()
noDepDelayedNAs.set_name(numCols,y_col)

In [8]:
gbm = h2o.gbm(x=noDepDelayedNAs[x_cols], y=noDepDelayedNAs[y_col], distribution="bernoulli")
gbm.show()


gbm Model Build Progress: [##################################################] 100%
Model Details
H2OBinomialModel :  Gradient Boosting Machine
Model Key:  GBM_model_python_1451948539188_362

Model Summary:



,number_of_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,34360.0,5.0,5.0,5.0,18.0,32.0,28.72




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.190998228325
R^2: 0.235487670482
LogLoss: 0.56424502622
AUC: 0.78729159221
Gini: 0.574583184419

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.416705112919:



,NO,YES,Error,Rate
NO,11468.0,9419.0,0.451,(9419.0/20887.0)
YES,3510.0,18495.0,0.1595,(3510.0/22005.0)
Total,14978.0,27914.0,0.3014,(12929.0/42892.0)



Maximum Metrics:



metric,threshold,value,idx
max f1,0.416705112919,0.741000420682,257.0
max f2,0.250036790849,0.847992818558,344.0
max f0point5,0.558092215426,0.728772266577,181.0
max accuracy,0.50917905367,0.713816096242,207.0
max precision,0.95217991806,1.0,0.0
max absolute_MCC,0.513158744745,0.427623630147,205.0
max min_per_class_accuracy,0.50917905367,0.712656214497,207.0



Scoring History:



,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_lift,training_classification_error
,2016-01-04 15:51:49,0.106 sec,0.0,0.249830147861,0.692807437805,0.5,1.0,0.486967266623
,2016-01-04 15:51:49,0.192 sec,1.0,0.244362718818,0.681856365041,0.692503203228,1.58804520775,0.413433740558
,2016-01-04 15:51:49,0.301 sec,2.0,0.239916651379,0.672915590687,0.700446640048,1.65506176274,0.410845845379
,2016-01-04 15:51:49,0.429 sec,3.0,0.235500532513,0.663968419721,0.712157526928,1.84660424066,0.391051944419
,2016-01-04 15:51:49,0.597 sec,4.0,0.231804609029,0.656396477796,0.717631801475,1.83993364737,0.387158444465
,2016-01-04 15:51:50,0.859 sec,5.0,0.228800636002,0.650232244088,0.725320553124,1.84541686654,0.377040007461
,2016-01-04 15:51:50,1.233 sec,6.0,0.225982520547,0.644333673011,0.727329858951,1.88213396038,0.380700363704
,2016-01-04 15:51:50,1.606 sec,7.0,0.22319866086,0.638455845729,0.7311981024,1.9165024911,0.37153781591
,2016-01-04 15:51:51,2.182 sec,8.0,0.220966984293,0.633646321473,0.732429446613,1.91636484531,0.36797071715
,2016-01-04 15:51:51,2.482 sec,9.0,0.21921730536,0.629926197946,0.735551823547,1.89921404809,0.357222792129



Variable Importances:



variable,relative_importance,scaled_importance,percentage
Origin,6802.29199219,1.0,0.408473570412
Dest,4523.11962891,0.664940528002,0.271610631585
DayofMonth,2025.07678223,0.297705065374,0.121604650983
UniqueCarrier,1249.49365234,0.183687153356,0.0750313473703
CRSArrTime,741.27154541,0.108973790931,0.0445129134631
CRSDepTime,640.993835449,0.0942320376993,0.0384912968862
DayOfWeek,422.001037598,0.0620380657111,0.0253409102025
CRSElapsedTime,138.01210022,0.0202890585083,0.00828754416917
Month,69.5189666748,0.0102199327454,0.00417457241789
Distance,41.1754722595,0.00605317623925,0.00247256251078
